In [ ]:
# 데이터 불러오기
import pandas as pd
import re 

pd.set_option('display.max_rows', 10000)

df = pd.read_csv(
    "./data/202512_202512_연령별인구현황_월간.csv",
    encoding="cp949"
)

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype("Int64")

# 3. 시/군 단위만 추출
# - 시도: code % 100_000_000 == 0
# - 시군구: code % 100_000_000 != 0 AND code % 100_000 == 0
sigungu = df[
    (df["행정코드"] % 100_000_000 != 0) &
    (df["행정코드"] % 100_000 == 0)
].copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2025년12월_계_총인구수",
    "2025년12월_계_20~24세",
    "2025년12월_계_25~29세",
    "2025년12월_계_30~34세",
]

for c in num_cols:
    sigungu[c] = (
        sigungu[c]
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(int)
    )

# 5. 청년(20~34) 인구 계산
sigungu["청년(20~34)인구"] = (
    sigungu["2025년12월_계_20~24세"]
    + sigungu["2025년12월_계_25~29세"]
    + sigungu["2025년12월_계_30~34세"]
)

sigungu["청년비율(20~34)"] = (
    sigungu["청년(20~34)인구"]
    / sigungu["2025년12월_계_총인구수"]
)

# 6. 제외할 광역 단위 정의
exclude_regions = [
    "서울특별시", "경기도", 
    "인천광역시", "광주광역시", "울산광역시",
    "부산광역시", "대구광역시", "대전광역시"
]

sigungu["상위지역"] = sigungu["지역명"].str.split().str[0]

local_sigungu = sigungu[
    ~sigungu["상위지역"].isin(exclude_regions) & (sigungu["지역명"].str.split().str.len() == 2)
].copy()

# 7. 결과 테이블 구성
result = local_sigungu[
    ["지역명", "2025년12월_계_총인구수", "청년(20~34)인구", "청년비율(20~34)"]
].rename(columns={"2025년12월_계_총인구수": "총인구"})

result["청년비율(20~34)"] = (result["청년비율(20~34)"] * 100).round(2)

# 8. 결과 출력
print("=== [지방 시/군] 청년비율(20~34) 높은 지역 TOP 20 ===")
display(result.sort_values("청년비율(20~34)", ascending=False).head(50))

print("=== [지방 시/군] 청년비율(20~34) 낮은 지역 TOP 20 ===")
display(result.sort_values("청년비율(20~34)", ascending=True).head(50))

print("=== [지방 시/군] 청년인구(20~34) 많은 지역 TOP 20 ===")
display(result.sort_values("청년(20~34)인구", ascending=False).head(50))

